# PODPAC AWS Support

In [ ]:
from podpac.managers import aws
from podpac import settings

## AWS Session

The session is used for authentication and setting the region of services

In [ ]:
# If no credentials are input, then PODPAC will look in the Settings 
settings['AWS_ACCESS_KEY_ID'] = 'id'
settings['AWS_SECRET_ACCESS_KEY'] = 'key'
session = aws.Session()
session.get_account_id()

In [ ]:
# If no credentials are input, and PODPAC settings are not set, then PODPAC looks for shared credentials stored by AWS cli
settings['AWS_ACCESS_KEY_ID'] = None
settings['AWS_SECRET_ACCESS_KEY'] = None
session = aws.Session()
session.get_account_id()

# Node Manager

In [ ]:
from podpac.managers import aws
from podpac import settings

# set logging to DEBUG to see build process
import logging
logger = logging.getLogger("podpac")
logger.setLevel(logging.DEBUG)

In [ ]:
node = aws.Lambda()
node

In [ ]:
# longer description
node.describe()

In [ ]:
# build function and all resources
# This can take up to 15-20 seconds if all resources need to be created
node.build()

In [ ]:
# example usage
import podpac
from podpac import clinspace

sin_coords_node = podpac.algorithm.SinCoords()
coordinates = podpac.Coordinates([clinspace(-90, 90, 180), clinspace(90,-90, 180), '2018-01-01'], ['lat', 'lon', 'time'])

node = aws.Lambda(source=sin_coords_node)
output = node.eval(coordinates)
output.plot()
pass

In [ ]:
# print logs
logs = node.get_logs()
logs

In [ ]:
# remove all resources
# node.delete(confirm=True)

### Build with customizations

In [ ]:
settings["FUNCTION_NAME"] = "podpac-lambda-autogen-local"
settings["S3_BUCKET_NAME"] = "podpac-test-autogen-10001"
settings["FUNCTION_ROLE_NAME"] = "podpac-lambda-autogen-local"

node = aws.Lambda()
node.describe()

In [ ]:
node.build()

In [ ]:
node.describe()

### Function utilities

In [ ]:
node.update_function()

### Build with restrictions

Restrict function evaluation to specific node definitions.

In [ ]:
# make node
import podpac
from podpac import clinspace
from podpac.managers import aws
from podpac import settings

# set logging to DEBUG to see build process
import logging
logger = logging.getLogger("podpac")
logger.setLevel(logging.DEBUG)

In [ ]:
settings["FUNCTION_NAME"] = "podpac-lambda-autogen-restricted"
settings["S3_BUCKET_NAME"] = "podpac-test-autogen-10001"
settings["FUNCTION_ROLE_NAME"] = "podpac-lambda-autogen-local"

# make node that we want to evaluate
sin_coords_node = podpac.algorithm.SinCoords()

# make lambda node that is restricted only to this node
node = aws.Lambda(function_restrict_pipelines=[sin_coords_node.hash])
node.describe()

In [ ]:
node.build()

In [ ]:
# eval at coordinates
coordinates = podpac.Coordinates([clinspace(-90, 90, 180), clinspace(90,-90, 180), '2018-01-01'], ['lat', 'lon', 'time'])

# this will work
node = aws.Lambda(source=sin_coords_node)
output = node.eval(coordinates)
output.plot()
pass

In [ ]:
# this should throw an error
sin_coords_node = podpac.algorithm.SinCoords(units="m")
node = aws.Lambda(source=sin_coords_node)
output = node.eval(coordinates)

# AWS Utilities

## S3

In [ ]:
from podpac.managers import aws

In [ ]:
session = aws.Session()
bucket = "podpac-test-bucket"
tags = {"owner": "creare"}

In [ ]:
aws.create_bucket(session, bucket, bucket_tags=tags)

In [ ]:
aws.get_bucket(session, bucket)

In [ ]:
# aws.delete_bucket(session, bucket)  # this works if no objects are in bucket

In [ ]:
aws.put_object(session, bucket, "test/object.ipynb", "nexrad.ipynb")

In [ ]:
aws.delete_bucket(session, bucket)  # this will fail with object in bucket

In [ ]:
aws.delete_bucket(session, bucket, delete_objects="True")  # this will fail with object in bucket

## IAM Roles

In [ ]:
from podpac.managers import aws

In [ ]:
session = aws.Session()
role_name = "podpac-test-role"
tags = {"owner": "creare"}

In [ ]:
role = aws.create_role(session, role_name, role_tags=tags)

In [ ]:
aws.get_role(session, role_name)

In [ ]:
# aws.delete_role(session, role_name)  # don't delete here since we will need this create a function

## Lambda Functions

In [ ]:
from podpac.managers import aws

In [ ]:
session = aws.Session()
function = "podpac-test-function"
function_role_arn = role['Arn']  # see IAM Roles above
function_handler = "handler.handler"
function_source_bucket = "podpac-dist"
function_source_key = "1.1.0/podpac_dist.zip"
tags = {"owner": "creare"}

In [ ]:
aws.create_function(session, 
                    function, 
                    function_role_arn=function_role_arn, 
                    function_handler=function_handler, 
                    function_source_bucket=function_source_bucket, 
                    function_source_key=function_source_key, 
                    function_tags=tags)

In [ ]:
aws.get_function(session, function)

In [ ]:
aws.delete_function(session, "podpac-test-function")

In [ ]:
# create from local zip file
function = "podpac-test-function-local"
aws.create_function(session,
                    function,
                    function_role_arn=function_role_arn, 
                    function_handler=function_handler,
                    function_zip_file="podpac_dist.zip",
                    function_tags=tags)

In [ ]:
aws.delete_function(session, function)

## API Gateway

In [ ]:
from podpac.managers import aws

In [ ]:
session = aws.Session()
api_name = "podpac-lambda-autogen-api"
api_endpoint = "eval"

In [ ]:
aws.get_api(session, api_name, api_endpoint)

In [ ]:
apigateway = session.client("apigateway")
response = apigateway.get_stages(restApiId="fs3mqqombi")

In [ ]:
apigateway.get_resources(restApiId="fs3mqqmbi")

## Get Logs

In [ ]:
from podpac.managers import aws

In [ ]:
session = aws.Session()
log_group_name = "/aws/lambda/podpac-test-function"

In [ ]:
aws.get_logs(session, log_group_name)